# 🔥 Fire...or No Fire?

## **Step 1:** Setup
We need to set up the dataframe to initial state.
Then we can manipulate it more in later steps, by cleaning it, digesting it, etc.

### Import Packages
> "If I have seen further, it is by standing on the shoulders of Giants."<br />
> &mdash; Isaac Newton


In [371]:
# glob crawler for reading files and folders
import glob

# regex parser for advanced string-parsing
import re

# pandas used for dataframe manipulation
import pandas

# numpy is used for odds and ends, like high-efficiency arrays
import numpy

# python imaging library; used for deconstructing images
from PIL import Image

# other imports are described as they are used
from keras.models import Sequential
from keras.losses import binary_crossentropy
from keras.optimizers import Adadelta
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

### Crawl Datafiles
The **glob** package is great and sets up our project for success.
Be sure you have your directory-structure set up correctly as stated in the README&hellip;
Otherwise, I cannot guarentee that this project will work for you!

In [372]:
# get all files from dirs in the data dir
files = glob.glob('data/*/*.*')

In [373]:
# prepare dataset in an array
dataset = []

# loop through every file that "glob" found.
for filepath in files:
	# regex used for Windows/MacOS compatibility
	filecrawl = re.split(r'\\+|/+', filepath)

	# remove the "data" folder entry; its not needed
	filecrawl = filecrawl[1:]

	# tag images from the fire-images folder as "fire"
	if filecrawl[0] == 'Fire images':
		filecrawl.append(1)
	else:
		filecrawl.append(0)

	# add filecrawl findings to dataset
	dataset.append(filecrawl)
	# ==NOTE==
	# Because the "glob" package arbitrarily crawls files, the
	# index of an item may not be the same each time this is run.

### Create Dataframe

In [374]:
# create our project's main dataframe
dataframe = pandas.DataFrame(dataset, columns=['folder', 'filename', 'fire'])

# display shows these tables neatly, shown below
display(dataframe.head(), dataframe.tail())

,folder,filename,fire
0,Fire images,1.jpg,1
1,Fire images,10-9-15-2-400.jpg,1
2,Fire images,11_10_19-mjs_ft_hotel-fire_19183862.jpg,1
3,Fire images,132-img1.png,1
4,Fire images,132343342_21n.jpg,1


,folder,filename,fire
646,Normal Images 5,x14862823.jpg,0
647,Normal Images 5,xbR1sVO.png,0
648,Normal Images 5,xR7vfcy.jpg,0
649,Normal Images 5,xsylvia-hotel-queen-room.jpg.pagespeed.ic.hw9T...,0
650,Normal Images 5,y5IQiH.jpg,0


## Step 2: Manipulation

### Checking for Duplicates

In [375]:
# check for any duplicate filenames in the dataframe
duplicates = dataframe['filename'].duplicated(keep=False)

# Select rows with duplicate filenames
duplicate_rows = dataframe[duplicates]
display(duplicate_rows)

# create a warning if duplicates exist
if duplicates.sum() != 0:
	warning = Warning(
		f'There are {duplicates.sum()}'
		' duplicated filenames in the dataframe.
		' proceed with caution.'
	) 
	print(warning)

,folder,filename,fire
0,Fire images,1.jpg,1
7,Fire images,14.jpg,1
70,Fire images,images.jpg,1
77,Fire images,maxresdefault.jpg,1
124,Normal Images 1,1.jpg,0
133,Normal Images 1,14.jpg,0
417,Normal Images 3,images.jpg,0
462,Normal Images 3,maxresdefault.jpg,0


There are 8 duplicated filenames in the dataframe. proceed with caution.


### Rebalancing Datapoints
I used [this article][rebalancing] to help figure things out.

[rebalancing]: https://towardsdatascience.com/having-an-imbalanced-dataset-here-is-how-you-can-solve-it-1640568947eb

In [376]:
# == FIXME ==
# this does not seem to be working properly at the moment

"""
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# create an object of the classifier, called "rebalancinator"
rebalancinator = BalancedBaggingClassifier(
	base_estimator=DecisionTreeClassifier(),
	sampling_strategy='auto',
	replacement=False,
	random_state=0
)

'''
# train the classifier.
rebalancinator.fit(x_train, y_train)
preds = rebalancinator.predict(x_train)
'''
"""

pass

### Train/Test Split
*Why do we need train/test split?*
*What does it do?*

In [377]:
# for x: keep the dataframe but drop the fire column
x = dataframe.drop(columns=['fire'])

# for y: drop everything in the dataframe but fire
y = dataframe.loc[:, ['fire']]

In [378]:
# Use the built-in "train test split" function
# to generate the four desireable segments of data.
x_train, x_test, y_train, y_test = train_test_split(
	x, y, test_size=0.35, random_state=0)

display(x_train.shape)
display(x_test.shape)

(423, 2)

(228, 2)

---

### Merge DataFrames

In [379]:
def get_img_vector(index):
	# find filepath
	filename = dataframe['filename'][index]
	folder = dataframe['folder'][index]
	filepath = f'data/{folder}/{filename}'

	# open the image via its filepath
	img = Image.open(filepath)
	# ==NOTE==
	# the Image class was imported from PIL
	# (python image library)

	# remove transparency layer
	img = img.convert('RGB')

	# resize the image
	img = img.resize((1024, 1024))

	# return the image vector
	return img_to_array(img)

In [380]:
def data_gen(dataframe, batch_size):
	# n_batch variables are empty arrays of constant size.
	# x_batch is has RGB values for each pixel's coordinate.
	# y_batch represents whether there is fire or not (0/1).
	x_batch = numpy.zeros((batch_size, 1024, 1024, 3))
	y_batch = numpy.zeros((batch_size, 1))

	# loop through entire dataframe, index by index
	for index in range(len(dataframe)):
		# digest the corresponding image into an array
		image_vector = get_img_vector(index)

		# using batch_size, we can determine 
		x_batch[index % batch_size] = get_img_vector(index)
		y_batch[index % batch_size] = dataframe['fire'][index]

		# if there has been {batch_size} items, we yield.
		# the last batch is an outlier; its batch is smaller.
		if ((index + 1) % batch_size == 0
		or (index + 1) == len(dataframe)):
			yield (x_batch, y_batch)

In [381]:
# dafuq does this mean
# its obviously important
model = Sequential()
# https://jovianlin.io/keras-models-sequential-vs-functional/

In [382]:
model.compile(
	loss=binary_crossentropy,
	optimizer=Adadelta(),
	metrics=['accuracy']
)

In [383]:
batch_size = 7
epochs = 3

model.fit_generator(
	generator = data_gen(
		x_train,
		batch_size = batch_size
	), 
	steps_per_epoch = len(x_train) // batch_size,
	epochs = epochs
)

RuntimeError: You must compile your model before using it.